# Credit Risk Resampling Techniques

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN


# Read the CSV into DataFrame

In [41]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [42]:
# Create our features
X = df.copy()
X.drop(columns=['loan_status'], axis=1, inplace=True)

# for scaler fit(), Perform a binary encoding on the 'homeowner' column using the Pandas get_dummies() function
X = pd.get_dummies(X, columns=['homeowner'])
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0


In [43]:
# Create our target
y = df['loan_status']
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [44]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [45]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [46]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [47]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [48]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [49]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [50]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)
Counter(y_train)

Counter({'low_risk': 56271, 'high_risk': 1881})

In [51]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [52]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[  563,    56],
       [  102, 18663]], dtype=int64)

In [53]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.91      0.99      0.88      0.95      0.90       619
   low_risk       1.00      0.99      0.91      1.00      0.95      0.91     18765

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [54]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_ros)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [55]:
# Train the Logistic Regression model using the resampled data
model_random = LogisticRegression(solver='lbfgs', random_state=1)
model_random.fit(X_resampled_ros, y_resampled_ros)

LogisticRegression(random_state=1)

In [56]:
# Calculated the balanced accuracy score
y_pred_ros = model_random.predict(X_test_scaled)
print('Balanced Accuracy Score:\n', balanced_accuracy_score(y_test, y_pred_ros))


Balanced Accuracy Score:
 0.9934649587814939


In [57]:
# Display the confusion matrix
print("Confusion Matrix:\n",confusion_matrix(y_test, y_pred_ros))

Confusion Matrix:
 [[  615     4]
 [  124 18641]]


In [58]:
# Print the imbalanced classification report
print('Imbalanced Classification Report:\n', classification_report_imbalanced(y_test, y_pred_ros))


Imbalanced Classification Report:
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [59]:
# Resample the training data with SMOTE
smote = SMOTE(random_state=1, sampling_strategy=1.0)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_smote)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [60]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(random_state=1)

In [61]:
# Calculated the balanced accuracy score
y_pred_smote = model_smote.predict(X_test_scaled)
print('Balanced Accuracy Score:')
balanced_accuracy_score(y_test, y_pred_smote)

Balanced Accuracy Score:


0.9934649587814939

In [62]:
# Display the confusion matrix
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_smote))

Confusion Matrix:
 [[  615     4]
 [  124 18641]]


In [63]:
# Print the imbalanced classification report
print('Imbalanced Classification Report:\n', classification_report_imbalanced(y_test, y_pred_smote))


Imbalanced Classification Report:
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [64]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_cc)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [65]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [66]:
# Calculate the balanced accuracy score
y_pred_cc = model_cc.predict(X_test_scaled)
print('Balanced Accuracy Score:',balanced_accuracy_score(y_test, y_pred_cc))


Balanced Accuracy Score: 0.9929503031930944


In [67]:
# Display the confusion matrix
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred_cc))

Confusion Matrix:
 [[  614     5]
 [  113 18652]]


In [68]:
# Print the imbalanced classification report
print('Imbalanced Classification Report:\n',classification_report_imbalanced(y_test, y_pred_cc))

Imbalanced Classification Report:
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [69]:
# Resample the training data with SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled_smoteenn, y_resampled_smoteenn = smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_smoteenn)

Counter({'high_risk': 55524, 'low_risk': 55937})

In [70]:
# Train the Logistic Regression model using the resampled data
# Train the Logistic Regression model using the resampled data
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1)
model_smoteenn.fit(X_resampled_smoteenn, y_resampled_smoteenn)

LogisticRegression(random_state=1)

In [71]:
# Calculate the balanced accuracy scorey_pred_smoteenn = model_smoteenn.predict(X_test_scaled)
y_pred_smoteenn = model_smoteenn.predict(X_test_scaled)
print('Balanced Accuracy Score:\n',balanced_accuracy_score(y_test, y_pred_smoteenn))


Balanced Accuracy Score:
 0.9934649587814939


In [72]:
# Display the confusion matrix
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred_smoteenn))


Confusion Matrix:
 [[  615     4]
 [  124 18641]]


In [73]:
# Print the imbalanced classification report
print('Imbalanced Classification Report:\n', classification_report_imbalanced(y_test, y_pred_smoteenn))


Imbalanced Classification Report:
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   Three models have the same best balanced accuracy score of 0.9934649587814939. These models are: Naive Random Oversampling, SMOTE Oversampling, and SMOTEENN Combination Sampling.

2. Which model had the best recall score?

    The Recall score 0.99 was the avg/total recall score for all the models. 

3. Which model had the best geometric mean score?

    All models had the same score of 0.99 for the geometric mean. 
    
    The performed models are: Simple Logistic Regression, Naive Random Oversampling, SMOTE Oversampling, Cluster Centroids Undersampling, and SMOTEENN Combination Sampling.
